In [34]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2
%store -r dm
dm = dm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [26]:
data_A = pd.read_csv("current_csv_files/data_A.csv")
data_B = pd.read_csv("current_csv_files/data_B.csv")
data_C = pd.read_csv("current_csv_files/data_C.csv")

X_A = data_A.drop(['target', 'date_forecast'], axis='columns')
X_B = data_B.drop(['target', 'date_forecast'], axis='columns')
X_C = data_C.drop(['target', 'date_forecast'], axis='columns')

y_A = data_A['target']
y_B = data_B['target']
y_C = data_C['target']

In [35]:
test_A = pd.read_parquet("../A/X_test_estimated.parquet")
test_B = pd.read_parquet("../B/X_test_estimated.parquet")
test_C = pd.read_parquet("../C/X_test_estimated.parquet")

test_A = test_A.drop(['date_calc', 'date_forecast'], axis='columns')
test_B = test_B.drop(['date_calc', 'date_forecast'], axis='columns')
test_C = test_C.drop(['date_calc', 'date_forecast'], axis='columns')

test_A['est'] = 1
test_B['est'] = 1
test_C['est'] = 1

In [36]:
# Average CV score on the training set was: -218261.99683292146
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    RandomForestRegressor(bootstrap=True, max_features=0.45, min_samples_leaf=1, min_samples_split=19, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(X_A, y_A)
results_A = exported_pipeline.predict(test_A)

exported_pipeline.fit(X_B, y_B)
results_B = exported_pipeline.predict(test_B)

exported_pipeline.fit(X_C, y_C)
results_C = exported_pipeline.predict(test_C)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LassoLarsCV was fitted with feature names
  warnings.warn(


ValueError: Input contains NaN.